# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

## Import the dataset with:

### mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Import dependencies

from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient

mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created

print(mongo.list_database_names())

['admin', 'artefacts', 'autosaurus', 'classDB', 'config', 'import_json', 'local', 'met', 'travel_db', 'travel_db2', 'uk_food']


In [4]:
# assign the uk_food database to a variable name

db = mongo["uk_food"]

In [5]:
# review the collections in our new database

print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection

pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f1fd0beb4f811ca47c90c9'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [7]:
# assign the collection to a variable

establishments = db["establishments"]

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data

new_restaurant = { "BusinessName":"Penang Flavours", "BusinessType":"Restaurant/Cafe/Canteen", 
                  "BusinessTypeID":"", "AddressLine1":"Penang Flavours", "AddressLine2":"146A Plumstead Rd", 
                  "AddressLine3":"London", "AddressLine4":"", "PostCode":"SE18 7DY", "Phone":"", "LocalAuthorityCode":"511", 
                  "LocalAuthorityName":"Greenwich", "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk", "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk", 
                  "scores": { "Hygiene":"", "Structural":"", "ConfidenceInManagement":"" }, 
                  "SchemeType":"FHRS", "geocode": { "longitude":"0.08384000", "latitude":"51.49014200" }, 
                  "RightToReply":"", "Distance":4623.9723280747176, "NewRatingPending":True }

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted

query_new_restaurant = {"BusinessName":"Penang Flavours"}
pprint(establishments.find_one(query_new_restaurant))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f1fdbbfd45dcad776dc01b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query_BusinessType = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType":1}
limit = 1

result = list(establishments.find(query_BusinessType,fields).limit(limit))

pprint(result)

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('63f1fd0beb4f811ca47c90c9')}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID

establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": "1"}})

In [13]:
# Confirm that the new restaurant was updated
query_updated_new_restaurant = {"BusinessName":"Penang Flavours"}
fields = {"BusinessName": 1, "BusinessTypeID": 1, "BusinessType":1}
pprint(establishments.find_one(query_updated_new_restaurant, fields))

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 '_id': ObjectId('63f1fdbbfd45dcad776dc01b')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"

query_dover = {"LocalAuthorityName": "Dover"}

print("Number of documents having LocalAuthorityName as Dover:", establishments.count_documents(query_dover))

Number of documents having LocalAuthorityName as Dover: 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"

establishments.delete_many({"LocalAuthorityName": "Dover"})

In [16]:
# Check if any remaining documents include Dover

print("Number of documents having LocalAuthorityName as Dover:", establishments.count_documents(query_dover))

Number of documents having LocalAuthorityName as Dover: 0


In [17]:
# Check that other documents remain with 'find_one'

establishments.find_one()

{'_id': ObjectId('63f1fd0beb4f811ca47c93b0'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude

establishments.update_many({}, [{'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}}])

In [19]:
# Change the data type from String to Decimal for latitude

establishments.update_many({}, [{'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}}])

In [20]:
# Check that the coordinates are now numbers

query_lat_long = {}
fields = {"geocode.longitude": 1, "geocode.latitude": 1}
pprint(establishments.find_one(query_lat_long, fields))

{'_id': ObjectId('63f1fd0beb4f811ca47c93b0'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
